In [17]:
import itertools
import math
import random

def sort_hand(hand):
  hand = list(hand)
  if hand[0] < hand[1]:
    hand[0], hand[1] = hand[1], hand[0]
  if hand[2] < hand[3]:
    hand[2], hand[3] = hand[3], hand[2]
  first, last = hand[:2], hand[2:]
  if first[0] < last[0]:
    first, last = last, first
  elif first[0] == last[0] and first[1] < last[1]:
    first, last = last, first
  return first + last

def compute_figgie(hand):
  num_cards = sum(hand)
  suit_deck_composition = [12, 8, 10, 10]
  combos = list(set(itertools.permutations(suit_deck_composition)))
  results = []
  for combo in combos:
    try:
        ways = [math.comb(combo[i], hand[i]) for i in range(len(hand))]
        total_ways = math.prod(ways)
        results.append(total_ways)
    except ValueError:
        results.append(0)

  sum_results = sum(results)
  if sum_results == 0:
      return [0,0,0,0]

  results = [x / sum_results for x in results]
  new_results = [0, 0, 0, 0]
  for i in range(len(results)):
    try:
        idx_of_main_suit = combos[i].index(12)
        new_results[idx_of_main_suit] += results[i]
    except ValueError:
        pass
  return new_results

def list_equal(list1, list2):
  if len(list1) != len(list2):
    return False
  for i in range(len(list1)):
    if list1[i] != list2[i]:
      return False
  return True

def compute_hand_probabilities(hands, num_cards_to_draw, num_simulations=10000):
  freqs = [0 for _ in range(len(hands))]
  deck = [1] * 12 + [2] * 8 + [3] * 10 + [4] * 10
  
  valid_simulations = 0
  for _ in range(num_simulations):
    drawn_cards = random.sample(deck, num_cards_to_draw)
    counts = [drawn_cards.count(i) for i in range(1, 5)]
    counts = sort_hand(counts)
    
    found_hand = False
    for i in range(len(hands)):
      if list_equal(hands[i], counts):
        freqs[i] += 1
        found_hand = True
        break
    if found_hand:
        valid_simulations +=1

  if valid_simulations == 0:
      return [0.0] * len(hands)

  freqs = [x / valid_simulations for x in freqs]
  return freqs

def generate_hands(num_cards_in_hand):
    hands_set = set()
    for i in range(min(num_cards_in_hand, 7) + 1):
        for j in range(min(num_cards_in_hand - i, 7) + 1):
            for k in range(min(num_cards_in_hand - i - j, 7) + 1):
                l = num_cards_in_hand - i - j - k
                if 0 <= l <= 7 :
                    hand = [i, j, k, l]
                    hand = sort_hand(hand)
                    hands_set.add(tuple(hand))
    
    sorted_hands = sorted(list(hands_set), reverse=True)
    return sorted_hands

def print_results(num_cards, hands, hand_probs, figgie_results_list):
    print(f"--- Results for {num_cards}-card hands ---")
    print(f"{'Hand':<12} | {'Figgie Probs (%)':<16} | {'Prob (%)':<20}")
    print("-" * (12 + 3 + 16 + 3 + 8))
    total_hand_prob = 0
    for i in range(len(hands)):
        hand_str = str(hands[i])
        results_str = ", ".join([f"{r:>2}" for r in figgie_results_list[i]])
        hand_prob_percent = hand_probs[i] * 100
        total_hand_prob += hand_prob_percent
        print(f"{hand_str:<12} | {'[' + results_str + ']':<16} | {hand_prob_percent:>4.1f}")
    print("-" * (12 + 3 + 16 + 3 + 8))

for num_cards_in_hand in [10, 8]:
    print(f"Processing {num_cards_in_hand}-card hands...")
    hands = generate_hands(num_cards_in_hand)
    if not hands:
        print(f"No valid hands generated for {num_cards_in_hand} cards. Skipping.")
        continue

    print(f"Generated {len(hands)} unique hands.")
    hand_probs = compute_hand_probabilities(hands, num_cards_in_hand)
    
    all_figgie_results = []
    for i, hand in enumerate(hands):
        figgie_calc_results = compute_figgie(list(hand)) 
        rounded_results = [int(round(r * 100, 0)) for r in figgie_calc_results]
        all_figgie_results.append(rounded_results)

    print_results(num_cards_in_hand, hands, hand_probs, all_figgie_results)
    print()


Processing 10-card hands...
Generated 40 unique hands.
--- Results for 10-card hands ---
Hand         | Figgie Probs (%) | Prob (%)            
------------------------------------------
(7, 3, 0, 0) | [70, 16,  7,  7] |  0.0
(7, 2, 1, 0) | [72, 12,  9,  7] |  0.1
(7, 1, 2, 0) | [72,  9, 12,  7] |  0.1
(7, 1, 1, 1) | [73,  9,  9,  9] |  0.2
(7, 0, 3, 0) | [70,  7, 16,  7] |  0.0
(7, 0, 2, 1) | [72,  7, 12,  9] |  0.1
(6, 4, 0, 0) | [56, 28,  8,  8] |  0.0
(6, 3, 1, 0) | [59, 21, 11,  8] |  0.2
(6, 2, 2, 0) | [60, 16, 16,  8] |  0.4
(6, 2, 1, 1) | [61, 16, 12, 12] |  0.5
(6, 1, 3, 0) | [59, 11, 21,  8] |  0.3
(6, 1, 2, 1) | [61, 12, 16, 12] |  1.3
(6, 0, 4, 0) | [56,  8, 28,  8] |  0.1
(6, 0, 3, 1) | [59,  8, 21, 11] |  0.3
(6, 0, 2, 2) | [60,  8, 16, 16] |  0.4
(5, 5, 0, 0) | [41, 41,  9,  9] |  0.0
(5, 4, 1, 0) | [46, 32, 13,  9] |  0.3
(5, 3, 2, 0) | [48, 25, 18, 10] |  1.3
(5, 3, 1, 1) | [48, 25, 13, 13] |  1.1
(5, 2, 3, 0) | [48, 18, 25, 10] |  1.5
(5, 2, 2, 1) | [49, 19, 19, 14] |